In [1]:
%%capture
!pip install 'torch==2.5.1' einops datasets jaxtyping sae_lens transformer_lens openai tabulate "nbformat>=4.2.0" umap-learn hdbscan eindex-callum git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python git+https://github.com/callummcdougall/sae_vis.git@callum/v3

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-k_6_ucy0
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-k_6_ucy0
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit 1e6129d08cae7af9242d9ab5d3ed322dd44b4dd3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/callummcdougall/sae_vis.git (to revision callum/v3) to /tmp/pip-req-build-_s_f4n28
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/sae_vis.git /tmp/pip-req-build-_s_f4n28
  Running command git checkout -b callum/v3 --track origin/callum/v3
  Switched to a new branch 'callum/v3'
  Branch 'callum/v3' set up to track remote branch 'callum/v3' from 'origin'.
  Resolved https://github.com/callummcdougall/sae_vis.git to commit 2e8e0742d481d582a2028cd4910673

In [3]:
%%capture
!pip install --upgrade torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 4.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1


# Imports

In [1]:
import gc
import itertools
import math
import os
import random
import sys
from collections import Counter
from copy import deepcopy
from dataclasses import dataclass
from functools import partial
from pathlib import Path
from typing import Any, Callable, Literal, TypeAlias

import einops
import numpy as np
import pandas as pd
import plotly.express as px
import requests
import torch as t
import openai
from datasets import load_dataset
from huggingface_hub import hf_hub_download
from IPython.display import HTML, IFrame, clear_output, display
from jaxtyping import Float, Int
from rich import print as rprint
from rich.table import Table
from sae_lens import (
    SAE,
    ActivationsStore,
    HookedSAETransformer,
    LanguageModelSAERunnerConfig,
    SAEConfig,
    SAETrainingRunner,
    upload_saes_to_huggingface,
)
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory
from sae_vis import SaeVisConfig, SaeVisData, SaeVisLayoutConfig
from tabulate import tabulate
from torch import Tensor, nn
from torch.distributions.categorical import Categorical
from torch.nn import functional as F
from tqdm.auto import tqdm
from transformer_lens import ActivationCache, HookedTransformer, utils
from transformer_lens.hook_points import HookPoint

In [2]:
from matplotlib import pyplot as plt
import seaborn as sns

Integrazione in VSCode

In [7]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
git_repo = 'https://github.com/alessandro-mv21/SAE-LLMThesis'

In [25]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade --quiet
from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared

In [26]:
launch_ssh_cloudflared(password)
init_git_cloudflared(repository_url=git_repo + ".git",
         personal_token=github_access_token,
         branch="main",
         email="valente.alessandro.m@gmail.com",
         username="alessandro-mv21")

Successfully cloned the repository in ./SAE-LLMThesis


# Funzioni varie

In [ ]:
import os
import sys

# Setup HF key


Funzione per fare un quadrato (per visualizzare graficamente gli embedding o le features)

In [ ]:
import math
def make_square(num: int) -> tuple[int, int]:
    """
    Trova i due fattori più vicini alla radice quadrata di un numero.

    Args:
        numero: Il numero intero di cui trovare i fattori.

    Returns:
        Una tupla contenente i due fattori più vicini alla radice quadrata.
    """

    if num < 1:
        raise ValueError("Il numero deve essere maggiore o uguale a 1.")

    radice_quadrata = math.sqrt(num)
    fattore_inferiore = math.floor(radice_quadrata)
    fattore_superiore = math.ceil(radice_quadrata)

    while num % fattore_inferiore != 0:
        fattore_inferiore -= 1

    while num % fattore_superiore != 0:
        fattore_superiore += 1

    return fattore_inferiore, fattore_superiore

Funzioni per vedere la memoria occupata

In [ ]:
import gc

import torch as t
from openai import OpenAIError
from tabulate import tabulate


def get_tensor_size(obj):
    size = 0
    if t.is_tensor(obj):
        size += obj.element_size() * obj.nelement()
    return size


def get_tensors_size(obj):
    if isinstance(obj, t.nn.Module):
        return sum(get_tensor_size(p) for p in obj.parameters())
    if hasattr(obj, "state_dict"):
        return sum(get_tensor_size(t) for t in obj.state_dict().values())
    return get_tensor_size(obj)


def get_device(obj):
    if t.is_tensor(obj):
        return str(obj.device)
    if isinstance(obj, t.nn.Module):
        try:
            return str(next(iter(obj.parameters())).device)
        except StopIteration:
            return "N/A"
    return "N/A"


def print_memory_status():
    t.cuda.synchronize()
    allocated = t.cuda.memory_allocated(0)
    total = t.cuda.get_device_properties(0).total_memory
    free = total - allocated
    print(f"Allocated: {allocated / 1024**3:.2f} GB")
    print(f"Total:  {total / 1024**3:.2f} GB")
    print(f"Free:  {free / 1024**3:.2f} GB")


def profile_pytorch_memory(namespace: dict, n_top: int = 10, filter_device: str = None):
    print_memory_status()

    object_sizes = {}
    for name, obj in namespace.items():
        try:
            obj_type = (
                type(obj).__name__
                if isinstance(obj, t.nn.Module)
                else f"Tensor {tuple(obj.shape)}"
                if t.is_tensor(obj)
                else None
            )
            if obj_type is None:
                continue
            device = get_device(obj)
            if filter_device and device != filter_device:
                continue
            size = get_tensors_size(obj)
            object_sizes[name] = (obj_type, device, size / (1024**3))
        except (OpenAIError, ReferenceError):
            # OpenAIError: we can't inspect the type of certain objects without triggering API request
            # ReferenceError: this object might have been garbage collected, so we don't care about it
            continue

    # Convert bytes to GB, sort by size & print
    sorted_sizes = sorted(object_sizes.items(), key=lambda x: x[1][2], reverse=True)[:n_top]
    table_data = [(name, obj_type, device, size) for name, (obj_type, device, size) in sorted_sizes]
    print(
        tabulate(
            table_data, headers=["Name", "Object", "Device", "Size (GB)"], floatfmt=".2f", tablefmt="simple_outline"
        )
    )


def find_cuda_tensors():
    cuda_tensors = []
    for obj in gc.get_objects():
        try:
            if t.is_tensor(obj) and obj.is_cuda:
                cuda_tensors.append(obj)
        except:
            pass
    return cuda_tensorsimport gc

import torch as t
from openai import OpenAIError
from tabulate import tabulate


def get_tensor_size(obj):
    size = 0
    if t.is_tensor(obj):
        size += obj.element_size() * obj.nelement()
    return size


def get_tensors_size(obj):
    if isinstance(obj, t.nn.Module):
        return sum(get_tensor_size(p) for p in obj.parameters())
    if hasattr(obj, "state_dict"):
        return sum(get_tensor_size(t) for t in obj.state_dict().values())
    return get_tensor_size(obj)


def get_device(obj):
    if t.is_tensor(obj):
        return str(obj.device)
    if isinstance(obj, t.nn.Module):
        try:
            return str(next(iter(obj.parameters())).device)
        except StopIteration:
            return "N/A"
    return "N/A"


def print_memory_status():
    t.cuda.synchronize()
    allocated = t.cuda.memory_allocated(0)
    total = t.cuda.get_device_properties(0).total_memory
    free = total - allocated
    print(f"Allocated: {allocated / 1024**3:.2f} GB")
    print(f"Total:  {total / 1024**3:.2f} GB")
    print(f"Free:  {free / 1024**3:.2f} GB")


def profile_pytorch_memory(namespace: dict, n_top: int = 10, filter_device: str = None):
    print_memory_status()

    object_sizes = {}
    for name, obj in namespace.items():
        try:
            obj_type = (
                type(obj).__name__
                if isinstance(obj, t.nn.Module)
                else f"Tensor {tuple(obj.shape)}"
                if t.is_tensor(obj)
                else None
            )
            if obj_type is None:
                continue
            device = get_device(obj)
            if filter_device and device != filter_device:
                continue
            size = get_tensors_size(obj)
            object_sizes[name] = (obj_type, device, size / (1024**3))
        except (OpenAIError, ReferenceError):
            # OpenAIError: we can't inspect the type of certain objects without triggering API request
            # ReferenceError: this object might have been garbage collected, so we don't care about it
            continue

    # Convert bytes to GB, sort by size & print
    sorted_sizes = sorted(object_sizes.items(), key=lambda x: x[1][2], reverse=True)[:n_top]
    table_data = [(name, obj_type, device, size) for name, (obj_type, device, size) in sorted_sizes]
    print(
        tabulate(
            table_data, headers=["Name", "Object", "Device", "Size (GB)"], floatfmt=".2f", tablefmt="simple_outline"
        )
    )


def find_cuda_tensors():
    cuda_tensors = []
    for obj in gc.get_objects():
        try:
            if t.is_tensor(obj) and obj.is_cuda:
                cuda_tensors.append(obj)
        except:
            pass
    return cuda_tensors

## Controllo memoria

In [ ]:
device = t.device("mps" if t.backends.mps.is_available() else "cuda" if t.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Profile memory usage, and delete gemma models if we've loaded them in
namespace = globals().copy() | locals()
profile_pytorch_memory(namespace=namespace, filter_device="cuda:0")



---



# Caricamento del modello finetuned